In [44]:
from __future__ import absolute_import, unicode_literals

import hashlib
import json
import time
import traceback
from pprint import pprint
import requests
from collections import defaultdict
from pymongo import MongoClient, ReturnDocument
from connection import ConnectionTask
c=ConnectionTask()
import re

import conf
class LvmamaTask():
    #: If :const:`True` the task is an abstract base class.
    abstract = True
    _pro = None
    _stg = None
    _dev = None

    @property
    def pro(self):
        if self._pro is None:
            _pro = MongoClient(conf.PRO_MONGO_HOST, replicaset=conf.REPLICASET_NAME)
            self._pro = _pro["boluome"]
        return self._pro

    @property
    def stg(self):
        if self._stg is None:
            _stg = MongoClient(conf.STG_MONGO_HOST, replicaset=conf.REPLICASET_NAME)
            self._stg = _stg["boluome"]
        return self._stg

    @property
    def dev(self):
        if self._dev is None:
            _dev = MongoClient(conf.DEV_MONGO_HOST, replicaset=conf.REPLICASET_NAME)
            self._dev = _dev["boluome"]
        return self._dev
l=LvmamaTask()

In [2]:
api_url = "http://api.lvmama.com/distributorApi/2.0/api/ticketProd/"
app_key = "BOLUOMI"
secret = "5ef1f365cae39119bd04b8afb35b134f"

In [61]:
# 调dpp.function
from path import Path
Path('/root/dpp/').cd()
from lvmama_worker import good_price_fetch
good_price_fetch.apply_async(args=[None,None,False,True],queue='dpp_lvmama_tasks')

<AsyncResult: 985be016-731b-496d-b1c4-02c92093cbb3>

In [163]:
def scenic_info_list(scenic_id):
    """获取单个景区的基本信息"""
    ts = int(time.time())
    sign = hashlib.md5('{}{}{}'.format(secret, ts, secret).encode('utf-8')).hexdigest()
    key_data = {
        'appKey': app_key,
        'messageFormat': 'json',
        'timestamp': ts,
        'sign': sign,
        'scenicId': scenic_id
    }
    url=api_url+'/scenicInfoList'
    resp_data = requests.get(url, params=key_data).json()
    return resp_data
scenic_info_list('10925367') 

{'scenicNameList': [{'placeCity': '<![CDATA[上海]]>',
   'placeCountry': '<![CDATA[中国]]>',
   'placeProvince': '<![CDATA[上海]]>',
   'scenicId': '10925367',
   'scenicName': '<![CDATA[测试POI变更周丽专用]]>'}],
 'state': {'code': '1000', 'message': '成功', 'solution': '接口访问成功'}}

In [4]:
def product_ids_fetch():
    for product in l.pro['menpiao_products'].find({},['id']):
        product_id=product['id']
        print(product_id)
        products_info(product_id)
    return 
# product_ids_fetch()

In [7]:
def products_info(product_ids):
    ts = int(time.time())
    sign = hashlib.md5('{}{}{}'.format(secret, ts, secret).encode('utf-8')).hexdigest()
    key_data = {
        'appKey': app_key,
        'messageFormat': 'json',
        'timestamp': ts,
        'sign': sign,
        'productIds': product_ids
    }
    url = api_url + "/productInfoList"
    try:
        resp_data = requests.get(url, params=key_data).json()
        product_list=resp_data['productList']
        product_data=product_list[0]
#         return product_data
    except KeyError :
        pprint(resp_data)
        return 

    def dict_none():
        return None
    data = defaultdict(dict_none, product_data)

    def clean_data(data):
        if isinstance(data, str)and ('<![CDATA' in data):
            return data[9:-3]
        return data
    return data
    scenic_info = {}
    scenic_info['id'] = clean_data(data['productId'])
    if not scenic_info['id']:
        return 'products info err'
    scenic_info['placeId'] = clean_data(data['placeId'])
    scenic_info['placeName'] = clean_data(data['placeName'])
    scenic_info['type'] = clean_data(data['productType'])
    scenic_info['status'] = clean_data(data['productStatus'])
    scenic_info['posts'] = clean_data(data['postList'])
    scenic_info['recommends'] = clean_data(data['recommendList'])
    scenic_info['bookingInfo'] = clean_data(data['bookingInfo'])
    scenic_info['intro'] = clean_data(data['introdution'])
    scenic_info['activities'] = clean_data(data['activities'])
    scenic_info['character'] = clean_data(data['characteristic'])
    scenic_info['playAttractions'] = clean_data(data['playAttractions'])
    scenic_info['productTheme'] = clean_data(data['productTheme'])
    scenic_info['serviceGuarantee'] = clean_data(data['serviceGuarantee'])
    scenic_info['images'] = clean_data(data['images'])
    l.dev['menpiao_products'].update_one({'id': scenic_info['id']}, {'$set': scenic_info}, upsert=True)
    l.stg['menpiao_products'].update_one({'id': scenic_info['id']}, {'$set': scenic_info}, upsert=True)
#     l.pro['menpiao_products'].update_one({'id': scenic_info['id']}, {'$set': scenic_info}, upsert=True)

    goods_ids=data['goodsIds']
    
    return good_info_fetch(goods_ids) if goods_ids else 'no goods_ids'

# products_info(585857)

In [164]:
resp_data=products_info("572989")
pprint(resp_data)

defaultdict(<function products_info.<locals>.dict_none at 0x7f1f218bc400>,
            {'bookingInfo': {'freePolicy': '<![CDATA[a. 6周岁（含）以下的儿童免票。]]>',
                             'offerCrowd': '<![CDATA[a. '
                                           '65周岁（含）以上的老人（凭身份证）享8折优惠。]]>'},
             'characteristic': ['<![CDATA[景区特色]]>'],
             'goodsIds': '6689866',
             'images': ['http://pic.lvmama.com//uploads/pc/place2/2016-06-16/cb0700dd-444d-4f9f-976c-ef8206e68b75.jpg',
                        'http://pic.lvmama.com//uploads/pc/place2/2016-06-16/55752afa-39c3-47e1-b53e-dbd361faca44.jpg',
                        'http://pic.lvmama.com//uploads/pc/place2/2016-06-16/0a115a65-6cfa-41fd-9cbe-7313d77c27d2.jpg'],
             'introdution': '<![CDATA[syuuIE8景点门票简介描述]]>',
             'placeId': 10925367,
             'placeName': {'mainName': '<![CDATA[测试POI变更周丽专用]]>'},
             'playAttractions': [{'playImages': ['http://pic.lvmama.com//uploads/pc/place2/2015-08-07/af1d

In [18]:
def good_info_fetch(goods_id):
    ts = int(time.time())
    sign = hashlib.md5('{}{}{}'.format(secret, ts, secret).encode('utf-8')).hexdigest()
    key_data = {
        'appKey': app_key,
        'messageFormat': 'json',
        'timestamp': ts,
        'sign': sign,
        'goodsIds': goods_id
    }
    url = api_url + "/goodInfoList"
    try:
        resp_data = requests.get(url, params=key_data).json()['goodsList']
        return resp_data
    except KeyError:
        return 'good no data'
    for item in resp_data:
        def dict_none():
            return None
        data = defaultdict(dict_none, item)
        pprint(data)
        continue
        def clean_data(data):
            if isinstance(data, str)and ('<![CDATA' in data):
                return data[9:-3]
            return data
        good_info = {}
        good_info['goodsId'] = clean_data(data['goodsId'])
        if not good_info['goodsId']:
            return 'good info err'

        good_info['productId'] = clean_data(data['productId'])
        good_info['standardName'] = clean_data(data['standardName'])
        good_info['goodsName'] = clean_data(data['goodsName'])
        good_info['goodsSort'] = clean_data(data['goodsSort'])
        good_info['paymentType'] = clean_data(data['paymentType'])
        good_info['ticketType'] = clean_data(data['ticketType'])
        good_info['adultTicket'] = clean_data(data['adultTicket'])
        good_info['childTicket'] = clean_data(data['childTicket'])
        good_info['goodsType'] = clean_data(data['goodsType'])
        good_info['certificate'] = clean_data(data['certificate'])
        good_info['status'] = clean_data(data['status'])
        good_info['rules'] = clean_data(data['rules'])
        good_info['otherRule'] = clean_data(data['otherRule'])
        good_info['effective'] = clean_data(data['effective'])
        good_info['minimum'] = clean_data(data['minimum'])
        good_info['maximum'] = clean_data(data['maximum'])
        good_info['costInclude'] = clean_data(data['costInclude'])
        good_info['passTimeLimit'] = clean_data(data['passTimeLimit'])
        good_info['notice'] = clean_data(data['notice'])
        good_info['importentPoint'] = clean_data(data['importentPoint'])
        good_info['refundRuleNotice'] = clean_data(data['refundRuleNotice'])
        good_info['booker'] = clean_data(data['booker'])
        good_info['traveller'] = clean_data(data['traveller'])
        good_info['costInclude'] = clean_data(data['costInclude'])
        good_info['needTicket'] = clean_data(data['needTicket'])
        good_info['visitAddress'] = clean_data(data['visitAddress'])
        good_info['importantNotice'] = clean_data(data['importantNotice'])
        good_info['limitation'] = clean_data(data['limitation'])
        good_info['Ticketdescription'] = clean_data(data['Ticketdescription'])
#         pprint(good_info)
#         l.dev['menpiao_goods_lvmama'].update_one({'goodsId': good_info['goodsId']}, {'$set': good_info}, upsert=True)
#         l.stg['menpiao_goods_lvmama'].update_one({'goodsId': good_info['goodsId']}, {'$set': good_info}, upsert=True)
#         l.pro['menpiao_goods_lvmama'].update_one({'goodsId': good_info['goodsId']}, {'$set': good_info}, upsert=True)
    return 
# good_info_fetch('3971646')

In [16]:
data=good_info_fetch("1787823")

In [7]:
def good_price_fetch(goods_id):
    ts = int(time.time())
    begin_date=time.strftime("%Y-%m-%d",time.localtime(ts))
    end_date=time.strftime("%Y-%m-%d",time.localtime(ts+60*60*24*30))
    sign = hashlib.md5('{}{}{}'.format(secret, ts, secret).encode('utf-8')).hexdigest()
    key_data = {
        'appKey': app_key,
        'messageFormat': 'json',
        'timestamp': ts,
        'sign': sign,
        'goodsIds': goods_id,
        'beginDate':begin_date,
        'endDate':end_date
    }
    url = api_url + "/goodPriceList"
    try:
        resp_data = requests.get(url, params=key_data).json()['priceList'][0]['goodsList']['goods']
        return resp_data
    except KeyError:
        return 'good price no data'
    pprint(resp_data)
    return
    for item in resp_data:
        def dict_none():
            return None
        data = defaultdict(dict_none, item)
        def clean_data(data):
            if isinstance(data, str)and ('<![CDATA' in data):
                return data[9:-3]
            return data
        good_info = {}
        good_info['goodsId'] = clean_data(data['goodsId'])
        if not good_info['goodsId']:
            return 'good info err'
        good_info['goodsId'] = clean_data(data['goodsId'])
        good_info['goodsName'] = clean_data(data['goodsName'])
        good_info['goodsOnLine'] = clean_data(data['goodsOnLine'])
        good_info['prices'] = clean_data(data['prices'].get('price'))
        
        l.dev['menpiao_goods_lvmama'].update_one({'goodsId': good_info['goodsId']}, {'$set': good_info}, upsert=True)
        l.stg['menpiao_goods_lvmama'].update_one({'goodsId': good_info['goodsId']}, {'$set': good_info}, upsert=True)
#         l.pro['menpiao_goods_lvmama'].update_one({'goodsId': good_info['goodsId']}, {'$set': good_info}, upsert=True)
    return 

In [30]:
l.dev['menpiao_goods'].update_one({'id':'1787823'},{'$set':{'prices': [{'aheadHour': 2880,
                        'date': '2017-07-09',
                        'marketPrice': 4900.0,
                        'sellPrice': 3380.0,
                        'stock': -1},
                       {'aheadHour': 2880,
                        'date': '2017-07-16',
                        'marketPrice': 4900.0,
                        'sellPrice': 3380.0,
                        'stock': -1}]}})

In [26]:
data=good_price_fetch('1787823')

## 2.全量更新数据库
+ 2.1 批量景区基本信息接口 scenicInfoListByPage

In [11]:
def scenic_info_fetch(current_page=1):
    ts = int(time.time())
    sign = hashlib.md5('{}{}{}'.format(secret, ts, secret).encode('utf-8')).hexdigest()
    key_data = {
        'appKey': app_key,
        'messageFormat': 'json',
        'timestamp': ts,
        'sign': sign,
        'currentPage': current_page
    }
    api_url = "http://api.lvmama.com/distributorApi/2.0/api/ticketProd/"
    url = api_url + "/scenicInfoListByPage"
    try:
        resp_data = requests.get(url, params=key_data).json()
        return resp_data
        scenic_name_list=resp_data['scenicNameList']
        total_page=resp_data['totalPage'] if current_page==1 else 0
#         product_data=product_list[0]
    except KeyError :
        pprint(resp_data)
        return 
    
    for item in scenic_name_list:
        def dict_none():
            return None
        data = defaultdict(dict_none, product_data)

        def clean_data(data):
            if isinstance(data, str)and ('<![CDATA' in data):
                return data[9:-3]
            return data

#         scenic_info = {}
#         scenic_info['id']=d
#         l.stg['menpiao_products'].update_one({'id': scenic_info['id']}, {'$set': scenic_info}, upsert=True)
    #     l.pro['menpiao_products'].update_one({'id': scenic_info['id']}, {'$set': scenic_info}, upsert=True)

        goods_ids=data['goodsIds']
    
    return 

resp_data=scenic_info_fetch(559)

In [ ]:
def products_info(product_ids):
    ts = int(time.time())
    sign = hashlib.md5('{}{}{}'.format(secret, ts, secret).encode('utf-8')).hexdigest()
    key_data = {
        'appKey': app_key,
        'messageFormat': 'json',
        'timestamp': ts,
        'sign': sign,
        'productIds': product_ids
    }
    url = api_url + "/productInfoList"
    try:
        resp_data = requests.get(url, params=key_data).json()
        product_list=resp_data['productList']
        product_data=product_list[0]
    except KeyError :
        pprint(resp_data)
        return 

    def dict_none():
        return None
    data = defaultdict(dict_none, product_data)

    def clean_data(data):
        if isinstance(data, str)and ('<![CDATA' in data):
            return data[9:-3]
        return data
    scenic_info = {}
    scenic_info['id'] = clean_data(data['productId'])
    if not scenic_info['id']:
        return 'products info err'
    scenic_info['placeId'] = clean_data(data['placeId'])
    scenic_info['placeName'] = clean_data(data['placeName'])
    scenic_info['type'] = clean_data(data['productType'])
    scenic_info['status'] = clean_data(data['productStatus'])
    scenic_info['posts'] = clean_data(data['postList'])
    scenic_info['recommends'] = clean_data(data['recommendList'])
    scenic_info['bookingInfo'] = clean_data(data['bookingInfo'])
    scenic_info['intro'] = clean_data(data['introdution'])
    scenic_info['activities'] = clean_data(data['activities'])
    scenic_info['character'] = clean_data(data['characteristic'])
    scenic_info['playAttractions'] = clean_data(data['playAttractions'])
    scenic_info['productTheme'] = clean_data(data['productTheme'])
    scenic_info['serviceGuarantee'] = clean_data(data['serviceGuarantee'])
    scenic_info['images'] = clean_data(data['images'])
    l.dev['menpiao_products'].update_one({'id': scenic_info['id']}, {'$set': scenic_info}, upsert=True)
    l.stg['menpiao_products'].update_one({'id': scenic_info['id']}, {'$set': scenic_info}, upsert=True)
#     l.pro['menpiao_products'].update_one({'id': scenic_info['id']}, {'$set': scenic_info}, upsert=True)

    goods_ids=data['goodsIds']
    
    return good_info_fetch(goods_ids) if goods_ids else 'no goods_ids'

# products_info(585857)

In [8]:
class test():
    def __init__(self):
        self._number=0
app=test()


### 统计驴妈妈goods 不支持分销情况

In [52]:
counter=set([])
for item in l.pro.menpiao_goods_1.find({"prices":{"$exists":True},"status":True},{"prices":1,"id":1}):
    if not item['prices']:
        continue
    for price in item['prices']:
        if price['stock'] == 0:
            counter.add(item["id"])
            break
print(counter)

set()


### lvmama 景区数据补偿

In [80]:
import json
import re
import ast
from connection import ConnectionTask
c=ConnectionTask()

In [48]:
with open('/data/scenic_name_data.json') as f:
    data=f.read()

In [84]:
count=0
err_item=[]
for line in  re.finditer('\{\"state\".*?\}\}\]\}', data):
    item=line.group()
    try:
        scenic_list=json.loads(item)
    except :
        err_item.append(item)
    for scenic in scenic_list['scenicNameList']:
        count+=1
        scenic = defaultdict(lambda :None, scenic)

        scenic_info = {}
        scenic_id = scenic['scenicId']
        if not scenic_id:
            continue
        scenic_info['status'] = True
        scenic_info['name'] = scenic['scenicName']
        scenic_info['info'] = scenic['placeInfo']
        scenic_info['act'] = scenic['placeAct']
        scenic_info['level'] = scenic['placeLevel']
        scenic_info['image'] = scenic['placeImage']
        scenic_info['addr'] = scenic['placeToAddr']
        scenic_info['openTimes'] = scenic['openTimes']
        scenic_info['town'] = scenic['placeTown']
        scenic_info['xian'] = scenic['placeXian']
        scenic_info['city'] = scenic['placeCity']
        scenic_info['province'] = scenic['placeProvince']
        scenic_info['country'] = scenic['placeCountry']
        scenic_info['baiduData'] = scenic['baiduData'] if scenic_id != "175649" else {"longitude": "121.6572887",
                                                                                    "latitude": "31.1452795"}
        scenic_info['googleData'] = scenic['googleData'] if scenic_id != "175649" else {"longitude": "121.6572887",
                                                                                      "latitude": "31.1452795"}
        loc_data = scenic_info['baiduData'] or scenic_info['googleData']
        scenic_info['loc'] = [float(loc_data['longitude']), float(loc_data['latitude'])] if loc_data else []
        scenic_info['__v'] =10086
        print(count,scenic_id,end='\r')
        c.smdb['menpiao_scenic'].update_one({'id': scenic_id}, {'$set': scenic_info}, upsert=True)
        c.static['menpiao_scenic'].update_one({'id': scenic_id}, {'$set': scenic_info}, upsert=True)

In [38]:
# lvmama.location
import ast

for item in c.smdb.menpiao_scenic.find():
    loc=ast.literal_eval(item['loc'])
    location={
        "type": "Point",
        "coordinates": loc
    } if loc else {}
#     item['location']=location
    c.smdb.menpiao_scenic.update_one({'id':item['id']},{'$set':{'location':location}})
#     print(item)
#     break

In [42]:
#play_image_url_fetch
for item in c.static.menpiao_products.find(
        {
            'playAttractions': {'$exists': True, '$ne': []},
            'status': True
        },
        {'id': 1, 'playAttractions': 1, '_id': 0}
):
    url_list = item.get('playAttractions', [])
    product_id = item.get('id', '')
    if not (url_list and product_id):
        continue
    status = True
    for url in url_list:
        if 'sslPlayImages' in url:
            status = False
            break
        else:
            continue
    if status:
        print(product_id)

In [38]:
from path import Path
Path('/root/dpp/').cd()
from lvmama_worker import file_transport_to_server

In [40]:
a=Path('/data/lvmama/')
for name in a.walkfiles('*.jpg'):
    file_transport_to_server.apply_async(args=[name],queue='dpp_lvmama_tasks')

In [28]:
class Demo(object):
    _ssh=None
    @property
    def ssh(self):
        if Demo._ssh is None:
            Demo._ssh='hello'
        return Demo._ssh
    @property
    def init(self):
        Demo._ssh=None

In [31]:
d=Demo()

In [33]:
d.ssh

'hello'

In [35]:
d.init

In [36]:
Demo._ssh

In [ ]:
import paramiko
ssh = paramiko.SSHClient()
            # 这行代码的作用是允许连接不在know_hosts文件中的主机。
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh.connect("139.198.3.124", username="root",timeout=3)
paramiko.SFTPClient.from_transport(ssh.get_transport())

In [155]:
sftp = ssh.open_sftp()
sftp2 = ssh.open_sftp()

In [140]:
ssh.get_transport().is_active()

True

In [149]:
sftp.close()

In [161]:
try:
    raise KeyError
    sftp.listdir('/data/aa')
except FileNotFoundError:
    print('2')
except Exception:
    print(3)

3


In [41]:
with open('/root/logs/lvmama.log') as f:
    lvmama_text=f.read()

In [55]:
product_id_set=set()
rcm=re.compile("productId:.*?\,")
for item in re.finditer(rcm,lvmama_text):
    product_id = item.group().strip(',').split(':')[-1]
    product_id_set.add(product_id)
print(product_id_set)

{'182043', '588613', '183302', '1054560', '842673', '245778', '595738', '433405', '251995', '252153', '238310', '242131', '591489', '1512633', '254528', '951301', '444263', '183215', '1509119', '1019030', '182690', '586908', '811208', '1198451', '181771', '1536640', '250311', '769379', '359680', '946081', '237362', '1501462', '183606', '253080', '1504389', '172440', '647355', '438216', '183515', '808048', '182722', '1526217', '183420', '251485', '248668', '1496859', '182645', '183306', '182515', '955221', '234726', '253986', '785414', '253366', '698844', '270023', '781146', '293301', '1558160', '252939', '873994', '181669', '364432', '781315', '183692', '1523315', '181696', '242805', '172170', '236224', '251878', '181844', '788849', '182137', '1533572', '877518', '663619', '1537345', '1054895', '248586', '253369', '255092', '181932', '331630', '183928', '182462', '181781', '252600', '253935', '253171', '234896', '1523762', '255989', '652419', '255292', '181746', '1550149', '252364', '1

In [59]:
from path import Path
Path('/root/dpp/').cd()
from lvmama_worker import good_price_fetch

In [60]:
for product_id in product_id_set:
    good_price_fetch.apply_async(args=[product_id], queue='dpp_lvmama_tasks')
    break

KeyboardInterrupt: 

In [42]:
lvmama_text[:1000]

"[2017-12-26 09:35:15,339: INFO/ForkPoolWorker-37] Task lvmama_worker.good_price_fetch[f785bb93-e582-4f0f-a1ee-0e832ed850fb] succeeded in 0.21321648359298706s: 'productId:182894,pdInfo:False'\n[2017-12-26 09:35:15,340: INFO/MainProcess] Received task: lvmama_worker.good_price_fetch[6e12cb9b-633d-4970-accb-77c81652b65b]  \n[2017-12-26 09:35:15,365: INFO/ForkPoolWorker-26] Task lvmama_worker.good_price_fetch[724461fa-2f3d-406e-bdf0-bac47ebab0a2] succeeded in 0.219253933057189s: 'productId:930512,pdInfo:False'\n[2017-12-26 09:35:15,365: INFO/ForkPoolWorker-24] Task lvmama_worker.good_price_fetch[59881343-859a-4893-9f12-8ea13184efbe] succeeded in 0.2191107403486967s: 'productId:172367,pdInfo:False'\n[2017-12-26 09:35:15,389: INFO/ForkPoolWorker-23] Task lvmama_worker.good_price_fetch[cfe8c3bd-2568-4c4a-820d-fdfc5a5807eb] succeeded in 0.23662403225898743s: 'productId:1523489,pdInfo:False'\n[2017-12-26 09:35:15,394: INFO/ForkPoolWorker-31] Task lvmama_worker.good_price_fetch[7a576013-c1e8-4a